In [ ]:
#Sentiment analysis program that parses the tweets fetched from
#twitter

In [1]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
# Twitter API credentials
key = "Y6e6vo4pw6sX2o3PtZhJQPNxB"
secretKey = "haYmIktsZbpu9fswSwKL7kxGQqstAi1vatGk9kMcIlBrdF7kxT"
token = "1324898678767824897-NDRINyeAuI6f64KsWwjDTx1nodRKBA"
secretToken = "UtS9eWfaRDKZti6L5wlPm661AYR7iORQTvr2HYXrdaCYW"

In [3]:
# log in and authenticate your twitter API
authenticate = tweepy.OAuthHandler(key, secretKey)
authenticate.set_access_token(token, secretToken)
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [52]:
text_query = 'Trump'
count = 150
try:
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.search,q=text_query).items(count)
 
 # Pulling information from tweets iterable object
 tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 tweets_df = pd.DataFrame(tweets_list)
 
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)
    
tweets_list

[[datetime.datetime(2020, 11, 14, 15, 55, 38),
  1327641348003012611,
  '@Robludlow859 @BMiddle888 @realDonaldTrump Generals, including the joint chiefs of staff, have already told trump t… https://t.co/XhVubNWiop'],
 [datetime.datetime(2020, 11, 14, 15, 55, 38),
  1327641347327750144,
  'RT @CNN: CNN PROJECTION: President-elect Joe Biden wins Georgia, flipping the traditionally red state, and Donald Trump wins North Carolina…'],
 [datetime.datetime(2020, 11, 14, 15, 55, 38),
  1327641347273134081,
  "RT @donwinslow: Trump's lawyers in PENNSYLVANIA &amp; ARIZONA quit today."],
 [datetime.datetime(2020, 11, 14, 15, 55, 38),
  1327641347097042946,
  'RT @funder: A Trump appointee just went off on Fox News and said this was the most secure election in US history. Amazing. https://t.co/NF8…'],
 [datetime.datetime(2020, 11, 14, 15, 55, 38),
  1327641346463723523,
  '@JohnR25776401 @EversFam @afneil I am calling them baseless because every lawsuit has been rejected except one whic… https://t

In [23]:
#tuckerPosts = api.user_timeline(screen_name = "TuckerCarlson", count=150, lang='en', tweet_mode='extended')
trumpPosts= api.user_timeline(screen_name = "realDonaldTrump", count=150, lang='en', tweet_mode='extended')

bidenPosts = api.user_timeline(screen_name = "JoeBiden", count=150, lang='en', tweet_mode='extended')

#for tweet in trumpPosts[0:5]:
    #print(tweet.full_text + "\n")
#print("-----------------------/n")
#for tweet in bidenPosts[0:5]:
    #print(tweet.full_text + "\n")
    

    
    
dfT = pd.DataFrame([tweet.full_text for tweet in trumpPosts], columns=['Tweets'])
dfB = pd.DataFrame([tweet.full_text for tweet in bidenPosts], columns=['Tweets'])

#dfT.head()
#dfB.head()

,Tweets
0,"I am the president-elect, but will not be pres..."
1,I am alarmed by the surge in reported COVID-19...
2,As the remnants of Tropical Storm Eta continue...
3,Today marks five years since the horrific ISIS...
4,I extend my deep condolences to the loved ones...


In [65]:
#clean the text
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub(r'#', '', text) # Removing '#' hash tag
    text = re.sub(r'RT[\s]+', '', text) # Removing RT
    text = re.sub(r'https?:\/\/\S+', '', text) # Removing hyperlink

dfT["Tweets"] = dfT["Tweets"].apply(cleanTxt)
dfB["Tweets"] = dfB["Tweets"].apply(cleanTxt)

dfT
printf("\n")
dfB

TypeError: expected string or bytes-like object

In [51]:
#create function to get subjectivity and polarity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#create new columns
dfT['Subjectivity'] = dfT['Tweets'].apply(getSubjectivity)
dfB['Subjectivity'] = dfB['Tweets'].apply(getSubjectivity)
dfT['Polarity'] = dfT['Tweets'].apply(getPolarity)
dfB['Polarity'] = dfB['Tweets'].apply(getPolarity)

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'NoneType'>

In [53]:
# plot wordcloud
trumpWords = ' '.join([twts for twts in dfT['Tweets']])
bidenWords = ' '.join([twts for twts in dfB['Tweets']])
wordCloudT = WordCloud(width = 500, height=300, random_state = 21, max_font_size=119).generate(allWords)
wordCloudB = WordCloud(width = 500, height=300, random_state = 21, max_font_size=119).generate(allWords)

plt.imshow(wordCloudT, interpolation='bilinear')
plt.axis("off")
plt.show()

TypeError: sequence item 0: expected str instance, NoneType found

In [58]:
#create function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return "Neutral"
    else:
        return 'Positive'
    

dfT["Analysis"] = dfT["Polarity"].apply(getAnalysis)
dfB["Analysis"] = dfB["Polarity"].apply(getAnalysis)

KeyError: 'Polarity'

In [59]:
#print all postive tweets
j=1
sortedDFT = dfT.sort_values(by=['Polarity'])
for i in range(0, sortedDFT.shape[0]):
    if(sortedDFT['Analysis'][i] == 'Positive'):
        print(str(j)+ ')' + sortedDFT['Tweets'][i])
        print()
        j+=1
print("-------------------------------------------------------------\n")
sortedDFB = dfB.sort_values(by=['Polarity'])
for i in range(0, sortedDFB.shape[0]):
    if(sortedDFB['Analysis'][i] == 'Positive'):
        print(str(j)+ ')' + sortedDFB['Tweets'][i])
        print()
        j+=1


KeyError: 'Polarity'

In [61]:
j=1
sortedDFT = dfT.sort_values(by=['Polarity'], ascending = 'False')
for i in range(o, sortedDFT.shape[0]):
    if(sortedDFT['Analysis'][i] == 'Negative'):
        print(str(j) + ')'+ sortedDFT['Tweets'][i])
        print()
        j+=1
        
sortedDFB = dfB.sort_values(by=['Polarity'], ascending = 'False')
for i in range(o, sortedDFB.shape[0]):
    if(sortedDFB['Analysis'][i] == 'Negative'):
        print(str(j) + ')'+ sortedDFB['Tweets'][i])
        print()
        j+=1



KeyError: 'Polarity'

In [63]:
#plot polarity and subjectivity
plt.figure(figsize=(8,6))
for i in range(0, dfT.shape[0]):
    plt.scatter(dfT['Polarity'][i], dfT['Sugjectivity'][i], color = 'Blue')
plt.title('Sentiment Analysis for Recent Donald Trump Tweets')
plt.xlabel('Polarity')
ply.ylabel('Subjectivity')
plt.show()

KeyError: 'Polarity'

<Figure size 576x432 with 0 Axes>

In [64]:
plt.figure(figsize=(8,6))
for i in range(0, dfB.shape[0]):
    plt.scatter(dfB['Polarity'][i], dfB['Sugjectivity'][i], color = 'Blue')
plt.title('Sentiment Analysis for Recent Joe Biden Tweets')
plt.xlabel('Polarity')
ply.ylabel('Subjectivity')
plt.show()

KeyError: 'Polarity'

<Figure size 576x432 with 0 Axes>